# Methodology and study procedure are described well. (2P)
Johannes

# Data of at least three participants is captured appropriatly. (3P)

already done

# Results are reported, compared, and discussed well. (2P)
Michael

# There is a section discussing problems that occurred during the assignment. (1P)
both